<a href="https://colab.research.google.com/github/kosior3x/atesty-/blob/main/Kopia_notatnika_Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

import os
import shutil
import subprocess
from google.colab import drive

# --- 1. STAŁE KONFIGURACYJNE ---
# Główne katalogi
DRIVE_ARDUINO_DIR = '/content/drive/MyDrive/Arduino_Colab_Config'
ARDUINO_CLI_BIN = '/content/bin/arduino-cli'
CONFIG_DIR = '/root/.arduino15'
DRIVE_BIN_PATH = os.path.join(DRIVE_ARDUINO_DIR, 'bin', 'arduino-cli')

# --- 2. FUNKCJE POMOCNICZE ---

def run_command(command, description):
    """ Uruchamia komendę z logowaniem i sprawdzeniem błędu. """
    print(f"\n🚀 {description}...")
    try:
        # Używamy shell=True, gdybyśmy potrzebowali skomplikowanych operacji (choć tutaj nie jest to konieczne)
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        print(f"   ✅ Sukces. Wynik:\n{result.stdout.strip()[:200]}...")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ BŁĄD: {description} nie powiodło się.")
        print(f"      STDERR:\n{e.stderr.strip()}")
        raise
    except FileNotFoundError:
        print(f"   ❌ BŁĄD: Nie znaleziono narzędzia. Czy ścieżka do '{command.split()[0]}' jest poprawna?")
        raise

def fix_esp32_permissions():
    """ Ustawia uprawnienia do wykonania dla WSZYSTKICH narzędzi binarnych ESP32 (rozwiązuje błąd 'Permission denied'). """
    print("\n   [KRYTYCZNA POPRAWKA] Naprawiam uprawnienia dla narzędzi binarnych ESP32...")

    # Lista katalogów zawierających pliki binarne, które tracą uprawnienia po kopiowaniu z Dysku
    tool_bases = [
        os.path.join(CONFIG_DIR, 'packages', 'esp32', 'tools', 'esptool_py'),
        os.path.join(CONFIG_DIR, 'packages', 'esp32', 'tools', 'xtensa-esp32-elf-gcc'), # Alternatywna ścieżka G++
        os.path.join(CONFIG_DIR, 'packages', 'esp32', 'tools', 'esp-x32'), # G++ i inne narzędzia
        os.path.join(CONFIG_DIR, 'packages', 'esp32', 'tools', 'mklittlefs'), # Narzędzie do LittleFS/SPIFFS
    ]

    files_fixed_count = 0

    for base_dir in tool_bases:
        if not os.path.exists(base_dir):
            continue

        # Szukamy rekurencyjnie wszystkich plików
        for root, _, files in os.walk(base_dir):
            for file_name in files:
                file_path = os.path.join(root, file_name)

                # Ustawiamy uprawnienia do wykonania (0o755) dla plików, które nie są typowymi plikami źródłowymi/tekstowymi
                if not file_name.endswith(('.txt', '.py', '.h', '.lib', '.a', '.map', '.o', '.md', '.json', '.yaml')):

                    current_mode = os.stat(file_path).st_mode
                    # Sprawdzamy, czy plik nie ma już uprawnień do wykonania (bit 0o100)
                    if not (current_mode & 0o100):
                        os.chmod(file_path, 0o755)
                        files_fixed_count += 1

    print(f"   ✅ Poprawka uprawnień zakończona. Naprawiono uprawnienia dla **{files_fixed_count}** plików binarnych.")

# --- 3. LOGIKA GŁÓWNA ---

# Krok 1: Montowanie Dysku Google
print("--- Krok 1: Montowanie Dysku Google ---")
drive.mount('/content/drive')
print("✅ Dysk Google zamontowany.")

# Krok 2: Przygotowanie katalogów
os.makedirs(os.path.dirname(ARDUINO_CLI_BIN), exist_ok=True)
os.makedirs(os.path.dirname(DRIVE_BIN_PATH), exist_ok=True)
print(f"✅ Przygotowano katalogi na Arduino CLI i konfigurację: {DRIVE_ARDUINO_DIR}")

# Krok 3: Wybór scenariusza (Czy CLI jest już zainstalowane na Dysku?)
if os.path.exists(DRIVE_BIN_PATH):
    print("\n--- Krok 3: SCENARIUSZ A (CLI ZNALEZIONE NA DYSKU) ---")

    # A. Kopiowanie Arduino CLI z Dysku do lokalnej VM
    print("🚀 A. Kopiuję arduino-cli z Dysku do lokalnego /content/bin/...")
    shutil.copy2(DRIVE_BIN_PATH, ARDUINO_CLI_BIN)
    os.chmod(ARDUINO_CLI_BIN, 0o755)
    print("✅ Skopiowano i ustawiono uprawnienia do wykonania dla lokalnego CLI.")

    # B. Tworzenie symbolicznego linku do trwałej konfiguracji na Dysku
    if os.path.exists(CONFIG_DIR):
        shutil.rmtree(CONFIG_DIR)

    # Tworzymy link symboliczny, aby wszystkie pakiety i biblioteki były zapisywane bezpośrednio na Dysku
    os.symlink(DRIVE_ARDUINO_DIR, CONFIG_DIR)
    print(f"✅ Utworzono trwały link symboliczy '{CONFIG_DIR}' do '{DRIVE_ARDUINO_DIR}'")

    # C. Aktualizacja konfiguracji (jeśli pakiety były instalowane wcześniej)
    try:
        run_command(f"{ARDUINO_CLI_BIN} core update-index --config-file {os.path.join(CONFIG_DIR, 'arduino-cli.yaml')}",
                    "Aktualizacja indeksów rdzeni (dla pewności)")

        # D. KRYTYCZNA NAPRAWA: Ustawienie uprawnień dla narzędzi skopiowanych z Dysku
        fix_esp32_permissions()

    except Exception as e:
        print(f"⚠️ OSTRZEŻENIE: Wystąpił błąd podczas aktualizacji indeksów lub naprawy uprawnień: {e}. Spróbujmy zainstalować pakiety ponownie.")
        # Jeśli aktualizacja/poprawka zawiodła, przechodzimy do instalacji pakietów poniżej (Krok 4).
        pass

else:
    print("\n--- Krok 3: SCENARIUSZ B (PIERWSZE URUCHOMIENIE) ---")

    # A. Pobieranie Arduino CLI
    run_command(f"wget -q -O {ARDUINO_CLI_BIN} https://downloads.arduino.cc/arduino-cli/arduino-cli_latest_Linux_64bit.tar.gz && tar -xzf {ARDUINO_CLI_BIN} -C {os.path.dirname(ARDUINO_CLI_BIN)}",
                "Pobieranie i rozpakowywanie Arduino CLI")

    # B. Ustawienie uprawnień i kopiowanie na Dysk
    os.chmod(ARDUINO_CLI_BIN, 0o755)
    shutil.copy2(ARDUINO_CLI_BIN, DRIVE_BIN_PATH)
    print("✅ Skopiowano Arduino CLI na Dysk Google.")

    # C. Inicjalizacja konfiguracji (tworzy pliki konfiguracyjne w tymczasowym folderze)
    os.makedirs(CONFIG_DIR, exist_ok=True)
    run_command(f"{ARDUINO_CLI_BIN} config init --config-file {os.path.join(CONFIG_DIR, 'arduino-cli.yaml')}",
                "Inicjalizacja konfiguracji CLI")

    # D. Kopiowanie konfiguracji na Dysk i tworzenie linku symbolicznego
    shutil.copytree(CONFIG_DIR, DRIVE_ARDUINO_DIR, dirs_exist_ok=True)
    shutil.rmtree(CONFIG_DIR)
    os.symlink(DRIVE_ARDUINO_DIR, CONFIG_DIR)
    print(f"✅ Skopiowano konfigurację na Dysk Google i utworzono link '{CONFIG_DIR}' do '{DRIVE_ARDUINO_DIR}'")

# Krok 4: Instalacja pakietów ESP32 (wykonuje się w obu scenariuszach, jeśli pakiety nie istnieją)
# Używamy ścieżki do lokalnego CLI, które jest zawsze poprawne po Kroku 3
cli_path = ARDUINO_CLI_BIN
config_file = os.path.join(CONFIG_DIR, 'arduino-cli.yaml')
esp32_package_dir = os.path.join(CONFIG_DIR, 'packages', 'esp32')

if not os.path.exists(esp32_package_dir):
    print("\n--- Krok 4: INSTALACJA PAKIETÓW ESP32 (Pakiety nie znalezione na Dysku) ---")

    # A. Dodanie URL menedżera płyt ESP32
    run_command(f"{cli_path} config add board_manager.additional_urls https://raw.githubusercontent.com/espressif/arduino-esp32/gh-pages/package_esp32_index.json --config-file {config_file}",
                "Dodawanie URL menedżera płyt ESP32")

    # B. Aktualizacja indeksów
    run_command(f"{cli_path} core update-index --config-file {config_file}",
                "Aktualizacja indeksów rdzeni")

    # C. Instalacja rdzenia esp32
    run_command(f"{cli_path} core install esp32:esp32 --config-file {config_file}",
                "Instalacja rdzenia ESP32")

    # D. KRYTYCZNA POPRAWKA: Ustawienie uprawnień dla nowo pobranych narzędzi
    fix_esp32_permissions()

print("\n\n#################################################")
print("🎉 KONIEC KONFIGURACJI: Komórka 1 Zakończona.")
print("Teraz możesz bezpiecznie uruchomić Komórkę 2 (Interfejs Gradio).")
print("#################################################")

--- Krok 1: Montowanie Dysku Google ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dysk Google zamontowany.
✅ Przygotowano katalogi na Arduino CLI i konfigurację: /content/drive/MyDrive/Arduino_Colab_Config

--- Krok 3: SCENARIUSZ A (CLI ZNALEZIONE NA DYSKU) ---
🚀 A. Kopiuję arduino-cli z Dysku do lokalnego /content/bin/...
✅ Skopiowano i ustawiono uprawnienia do wykonania dla lokalnego CLI.
✅ Utworzono trwały link symboliczy '/root/.arduino15' do '/content/drive/MyDrive/Arduino_Colab_Config'

🚀 Aktualizacja indeksów rdzeni (dla pewności)...
   ✅ Sukces. Wynik:


   [KRYTYCZNA POPRAWKA] Naprawiam uprawnienia dla narzędzi binarnych ESP32...
   ✅ Poprawka uprawnień zakończona. Naprawiono uprawnienia dla **958** plików binarnych.


#################################################
🎉 KONIEC KONFIGURACJI: Komórka 1 Zakończona.
Teraz możesz bezpiecznie uruchomić Komórkę 2 (Interfejs Gradio).
#############

In [14]:

# --- 0. Konfiguracja i Importy ---
import gradio as gr
import os
import subprocess
import re
import shutil
from IPython.display import FileLink
import time

# Definicje stałych
DRIVE_ARDUINO_DIR = '/content/drive/MyDrive/Arduino_Colab_Config'
ARDUINO_CLI_BIN = '/content/bin/arduino-cli'
DRIVE_BIN_PATH = os.path.join(DRIVE_ARDUINO_DIR, 'bin', 'arduino-cli')
CONFIG_DIR = '/root/.arduino15'

SKETCH_NAME = "Colab_Sketch"
SKETCH_DIR = f'/content/{SKETCH_NAME}'
COMPILED_DIR = '/content/Compiled_Binaries'

# Mapa FQBN
FQBN_OPTIONS = {
    "ESP32 Dev Module (Ogólne)": "esp32:esp32:esp32",
    "ESP32 Wrover Module": "esp32:esp32:esp32wrover",
    "ESP32-S3 Dev Module": "esp32:esp32:esp32s3",
    "ESP32-C3 Dev Module": "esp32:esp32:esp32c3",
}

# Globalna zmienna przechowująca wyniki wyszukiwania bibliotek
SEARCH_RESULTS = {}

# --- 1. FUNKCJE INFRASTRUKTURALNE ---

def get_cli_path():
    """ Wybiera automatycznie ścieżkę do arduino-cli (Lokalna VM -> Dysk Google). """
    if os.path.exists(ARDUINO_CLI_BIN) and os.access(ARDUINO_CLI_BIN, os.X_OK):
        return ARDUINO_CLI_BIN
    if os.path.exists(DRIVE_BIN_PATH) and os.access(DRIVE_BIN_PATH, os.X_OK):
        return DRIVE_BIN_PATH
    raise FileNotFoundError(
        "❌ BŁĄD KRYTYCZNY: Nie znaleziono pliku 'arduino-cli' ani w lokalnej maszynie VM, ani na Dysku Google. Uruchom ponownie Komórkę 1."
    )

def get_cli_status_message():
    """ Zwraca informacyjny status CLI dla interfejsu. """
    try:
        path = get_cli_path()
        if path == ARDUINO_CLI_BIN:
            return f"✅ **Lokalny CLI** jest używany. (Szybsza ścieżka VM)"
        elif path == DRIVE_BIN_PATH:
            return f"⚠️ **Awaryjny CLI z Dysku Google** jest używany. (Ścieżka Drive)"
        return f"❓ Nieznany status CLI."
    except FileNotFoundError as e:
        return f"❌ **BŁĄD KRYTYCZNY:** {str(e)}"

# --- 2. FUNKCJE MENEDŻERA BIBLIOTEK ---

def search_libraries(query):
    """ Wyszukuje biblioteki i zwraca listę czytelnych opcji. """
    global SEARCH_RESULTS
    SEARCH_RESULTS = {}

    # 1. Weryfikacja CLI
    try:
        CLI_PATH = get_cli_path()
    except FileNotFoundError as e:
        return [str(e)], "Błąd CLI: Uruchom Komórkę 1."

    if not query or len(query.strip()) < 3:
        return [], "Wpisz co najmniej 3 znaki, aby wyszukać bibliotekę."

    # 2. Wykonanie wyszukiwania
    search_cmd = [
        CLI_PATH, 'lib', 'search', query,
        '--config-file', os.path.join(CONFIG_DIR, 'arduino-cli.yaml'),
        '--format', 'json' # Prosimy o JSON dla łatwiejszego parsowania
    ]

    print(f"\n--- Wyszukiwanie bibliotek dla: {query} ---")
    result = subprocess.run(search_cmd, capture_output=True, text=True)

    if result.returncode != 0:
        error_output = result.stderr.strip()
        print("--- Wyszukiwanie ZAKOŃCZONE BŁĘDEM ---")
        return [f"❌ BŁĄD WYSZUKIWANIA:\n{error_output}"], ""

    # 3. Parsowanie wyników
    try:
        import json
        data = json.loads(result.stdout)

        # Formatowanie wyników do czytelnej listy dla Gradio
        readable_options = []
        for item in data:
            name = item.get('name', 'N/A')
            author = item.get('author', 'Anonim')
            sentence = item.get('sentence', 'Brak opisu.')
            version = item.get('version', 'N/A')

            # Tworzymy czytelny format
            friendly_name = f"Nazwa: {name} | Wersja: {version} | Twórca: {author} | Opis: {sentence}"

            # Zapisujemy techniczną nazwę (lib_name) dla instalacji
            SEARCH_RESULTS[friendly_name] = name
            readable_options.append(friendly_name)

        if not readable_options:
            return [], "Nie znaleziono bibliotek pasujących do zapytania."

        return readable_options, f"Znaleziono {len(readable_options)} bibliotek. Wybierz jedną lub więcej do instalacji."

    except Exception as e:
        return [f"❌ BŁĄD PARSOWANIA WYNIKÓW: {e}"], ""


def install_selected_libraries(selected_libs):
    """ Instaluje wybrane biblioteki z listy wyników. """

    if not selected_libs:
        return "⚠️ OSTRZEŻENIE: Nie wybrano żadnej biblioteki do instalacji.", ""

    # 1. Weryfikacja CLI
    try:
        CLI_PATH = get_cli_path()
    except FileNotFoundError as e:
        return f"❌ BŁĄD: {str(e)}", ""

    successful = []
    failed = []

    # 2. Instalacja każdej wybranej biblioteki
    for friendly_name in selected_libs:
        lib_name = SEARCH_RESULTS.get(friendly_name)

        if not lib_name:
            failed.append(f"Nieznana nazwa techniczna dla '{friendly_name}'.")
            continue

        print(f"\n--- Rozpoczynam instalację: {lib_name} ---")

        install_cmd = [
            CLI_PATH, 'lib', 'install', lib_name,
            '--config-file', os.path.join(CONFIG_DIR, 'arduino-cli.yaml')
        ]

        result = subprocess.run(install_cmd, capture_output=True, text=True)

        if result.returncode == 0:
            successful.append(lib_name)
        else:
            failed.append(f"BŁĄD instalacji {lib_name}: {result.stderr.strip().splitlines()[0]}")

    # 3. Zwrócenie zbiorczego wyniku
    summary = ""
    if successful:
        summary += f"✅ **Zainstalowano pomyślnie:**\n* " + "\n* ".join(successful)
    if failed:
        summary += "\n\n❌ **Błędy instalacji:**\n* " + "\n* ".join(failed)

    return summary or "Zakończono. Brak nowych instalacji.", ""

# --- 3. FUNKCJE KOMPILACJI ---

def compile_and_get_bin(sketch_code, friendly_board_name):

    # 1. Weryfikacja ścieżki CLI
    try:
        CLI_PATH = get_cli_path()
    except FileNotFoundError as e:
        return f"❌ BŁĄD: {str(e)}", "BŁĄD"

    # 2. Mapowanie nazwy płytki
    board_fqbn = FQBN_OPTIONS.get(friendly_board_name)
    if not board_fqbn:
         return f"❌ BŁĄD: Niepoprawny wybór płytki.", "BŁĄD"

    # 3. Przygotowanie plików
    if not sketch_code.strip():
        return "BŁĄD: Wklej kod szkicu do pola tekstowego.", "BŁĄD"

    if os.path.exists(SKETCH_DIR):
        shutil.rmtree(SKETCH_DIR)

    os.makedirs(SKETCH_DIR, exist_ok=True)
    os.makedirs(COMPILED_DIR, exist_ok=True)

    sketch_file = os.path.join(SKETCH_DIR, f"{SKETCH_NAME}.ino")

    with open(sketch_file, 'w') as f:
        f.write(sketch_code)

    # 4. Kompilacja
    compile_cmd = [
        CLI_PATH, 'compile',
        '--fqbn', board_fqbn,
        '--config-file', os.path.join(CONFIG_DIR, 'arduino-cli.yaml'),
        '--build-path', COMPILED_DIR,
        SKETCH_DIR
    ]

    # Używamy subprocess.Popen, aby symulować konsolę w czasie rzeczywistym
    process = subprocess.Popen(
        compile_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    output_lines = []

    # Przetwarzanie wyjścia stdout w pętli
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            output_lines.append(line.strip())
            # Zwracamy postęp do konsoli (używamy yield, jeśli to byłby generator Gradio, ale w standardowej funkcji to tylko zbiór)
            print(line.strip())

    # Zbieranie błędów ze stderr (po zakończeniu)
    stderr_output = process.stderr.read()
    output_lines.extend(stderr_output.strip().splitlines())

    full_console_output = "\n".join(output_lines)

    # Sprawdzanie kodu wyjścia
    if process.returncode != 0:
        print("\n--- Kompilacja ZAKOŃCZONA BŁĘDEM ---")
        return f"❌ BŁĄD KOMPILACJI:\n\n{full_console_output}", "BŁĄD", f"Komenda: {' '.join(compile_cmd)}\n\n{full_console_output}"

    # 5. Wynik
    bin_file_name = f"{SKETCH_NAME}.ino.bin"
    bin_file_path = os.path.join(COMPILED_DIR, bin_file_name)

    if not os.path.exists(bin_file_path):
        return f"⚠️ OSTRZEŻENIE: Kompilacja zakończona sukcesem, ale nie znaleziono pliku {bin_file_name}.", "Plik binarny nieznaleziony", full_console_output

    print("--- Kompilacja ZAKOŃCZONA SUKCESEM ---")

    time.sleep(1)
    download_link = FileLink(bin_file_path).html_link
    file_size = os.path.getsize(bin_file_path) / (1024*1024)

    success_message = f"✅ KOMPILACJA ZAKOŃCZONA SUKCESEM!\n\n**Wybrana płytka:** **{friendly_board_name}** ({board_fqbn})\n**Rozmiar pliku binarnego:** **{file_size:.2f} MB**\n\nLink do pobrania pliku {bin_file_name}:\n{download_link}"

    return success_message, f"Sukces ({file_size:.2f} MB)", full_console_output


# --- 4. INTERFEJS GRADIO (Modułowy) ---

with gr.Blocks(title="Kompilator ESP32 Colab") as interface:
    gr.Markdown("## 🤖 Kompilator Arduino CLI (ESP32) w Google Colab")

    # --- PANEL STATUSU ---
    gr.Markdown("### ⚙️ Status Systemu")
    gr.Markdown(value=get_cli_status_message(), label="Aktualna Ścieżka Arduino CLI")
    gr.Markdown("---")

    # =================================================================
    # --- SEKCJA A: MENEDŻER BIBLIOTEK (Osobne okno) ---
    # =================================================================
    with gr.Tab("📚 Menedżer Bibliotek (Wyszukiwanie i Instalacja)"):
        gr.Markdown("### **Osobny Menedżer Bibliotek**")

        with gr.Row():
            search_query_input = gr.Textbox(label="Wyszukaj bibliotekę (np. FastLED)", placeholder="Wpisz co najmniej 3 znaki...")
            search_button = gr.Button("🔍 Szukaj")

        lib_status_output = gr.Markdown(value="Status wyszukiwania/instalacji pojawi się tutaj.", visible=True)

        # Pole do wyświetlania wyników i zaznaczania
        library_results_dropdown = gr.CheckboxGroup(
            label="Wyniki wyszukiwania (Zaznacz wybrane do instalacji):",
            choices=[],
            type="value",
            interactive=True,
            scale=4
        )

        install_button = gr.Button("🚀 Zainstaluj Zaznaczone Biblioteki")

        # Logika wyszukiwania
        search_button.click(
            fn=search_libraries,
            inputs=[search_query_input],
            outputs=[library_results_dropdown, lib_status_output]
        )

        # Logika instalacji
        install_button.click(
            fn=install_selected_libraries,
            inputs=[library_results_dropdown],
            outputs=[lib_status_output, library_results_dropdown] # Wyczyść listę po instalacji
        )

    # =================================================================
    # --- SEKCJA B: KOMPILACJA I KONSOLA ---
    # =================================================================
    with gr.Tab("🛠️ Kompilacja Szkicu i Konsola"):
        gr.Markdown("### **Kompilator Arduino**")

        with gr.Row():
            # Lewa strona: Kod i wybór płytki
            with gr.Column(scale=1):
                sketch_input = gr.Textbox(
                    label="Wklej tutaj kod szkicu Arduino (.ino)",
                    lines=15,
                    placeholder="void setup() {\n  Serial.begin(115200);\n}\n\nvoid loop() {\n  Serial.println(\"Hello, Colab!\");\n  delay(1000);\n}",
                    value="void setup() {\n  Serial.begin(115200);\n}\n\nvoid loop() {\n  Serial.println(\"Hello, Colab!\");\n  delay(1000);\n}"
                )

                friendly_board_dropdown = gr.Dropdown(
                    label="Wybierz płytkę ESP32 (Czytelna nazwa):",
                    choices=list(FQBN_OPTIONS.keys()),
                    value="ESP32 Dev Module (Ogólne)",
                    type="value"
                )

                compile_button = gr.Button("⚙️ KOMPILUJ KOD", variant="primary")

            # Prawa strona: Konsola i Status
            with gr.Column(scale=1):
                compilation_output = gr.Markdown(label="Status i Wynik Kompilacji")

                console_output = gr.Textbox(
                    label="Konsola Arduino CLI (Postęp, Ostrzeżenia, Błędy)",
                    lines=10,
                    interactive=False,
                    placeholder="Wszystkie komunikaty z konsoli Arduino CLI pojawią się tutaj."
                )

        # Ukryte pole (dla wewnętrznego statusu)
        short_status = gr.Textbox(label="Status Krótki", type="text", visible=False)

        # Logika kompilacji
        compile_button.click(
            fn=compile_and_get_bin,
            inputs=[sketch_input, friendly_board_dropdown],
            outputs=[compilation_output, short_status, console_output]
        )

# Uruchomienie interfejsu (dostępne pod linkiem publicznym)
interface.launch(debug=False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81d0d9591df5c5f200.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
